In [1]:
import tensorflow as tf
import numpy as np

In [2]:
num_classes = 10
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)


(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
y_test = tf.keras.utils.to_categorical(y_test)
y_train = tf.keras.utils.to_categorical(y_train)

In [3]:
x = tf.placeholder(tf.float32, shape = (None, img_rows, img_cols, 1))
y = tf.placeholder(tf.float32, shape = (None, num_classes))
dropout_probability = tf.placeholder(tf.float32)

conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=[3, 3], padding="valid", activation=tf.nn.relu)
pool2d = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
dropout_conv = tf.layers.dropout(pool2d, rate=dropout_probability, training=True)
conv1_flat = tf.layers.flatten(dropout_conv)
dense1 = tf.layers.dense(inputs=conv1_flat, units=128, activation=tf.nn.relu)
dropout_dense1 = tf.layers.dropout(dense1, rate=dropout_probability, training=True)
output = tf.layers.dense(inputs=dropout_dense1, units=10, activation=tf.nn.softmax)

In [4]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=output, logits=y))
# Same values as Keras defaults
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [5]:
def batch_data(source, target, batch_size):
   # Shuffle data
    shuffle_indices = np.random.permutation(np.arange(len(target)))
    source = source[shuffle_indices]
    target = target[shuffle_indices]
    while True:
        for batch_i in range(0, len(source)//batch_size):
            start_i = batch_i * batch_size
            source_batch = source[start_i:start_i + batch_size]
            target_batch = target[start_i:start_i + batch_size]

            yield np.array(source_batch), np.array(target_batch)

In [7]:
batch_size = 20
num_steps = 10**4
batch_generator = batch_data(x_train, y_train, batch_size)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(num_steps):
        batch_x, batch_y = next(batch_generator)
        sess.run(train_step, feed_dict={x: batch_x, y: batch_y, dropout_probability: 0.5})
        
    train_accuracy = accuracy.eval(feed_dict={x: x_test, y: y_test, dropout_probability: 0})
    print("step %d, training accuracy %g"%(i, train_accuracy))

step 9999, training accuracy 0.9764


## Same code with TensorBoard summaries:

While this code is running, you can monitor training using:

    tensorboard --logdir=eval/

In [8]:
tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()

with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)
    sess.run(tf.global_variables_initializer())

    for i in range(num_steps):
        batch_x, batch_y = next(batch_generator)
        summary, _ = sess.run([merged_summary, train_step], feed_dict={x: batch_x, y: batch_y, dropout_probability: 0.5})
        writer.add_summary(summary, i)
        
    train_accuracy = accuracy.eval(feed_dict={x: x_test, y: y_test, dropout_probability: 0.})
    print("step %d, training accuracy %g"%(i, train_accuracy))

step 9999, training accuracy 0.9787
